In [1]:
import torch

from src.model import get_model
from src.dataset import FrameDataset, split_dataset, get_pandas_dataset
from src.trainer import Trainer

In [2]:
baseTrainPath = 'data/train_images/train_images/'
dataset_path = 'data/usdc_train.json'

In [3]:
pandas_dataset, classes = get_pandas_dataset(dataset_path)

frame_dataset = FrameDataset(pandas_dataset, baseTrainPath)
train_dataset, test_dataset = split_dataset(frame_dataset)

pin_memory = torch.cuda.is_available()

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=lambda x: x,
    pin_memory=pin_memory)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    collate_fn=lambda x: x,
    pin_memory=pin_memory)

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = get_model(len(classes)).to(device)
optimizer = torch.optim.SGD(filter(lambda param: param.requires_grad, model.parameters()),
                            lr=0.001, momentum=0.9, weight_decay=0.0005)
num_epochs = 5

trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device
)

trainer.train()

Epoch 0: 100%|██████████| 464/464 [03:00<00:00,  2.57it/s]


Epoch 0: train_loss 0.641223132610321


Epoch 1: 100%|██████████| 464/464 [03:06<00:00,  2.49it/s]


Epoch 1: train_loss 0.5313564538955688


Epoch 2: 100%|██████████| 464/464 [03:12<00:00,  2.42it/s]


Epoch 2: train_loss 0.5143812298774719


Epoch 3: 100%|██████████| 464/464 [03:12<00:00,  2.41it/s]


Epoch 3: train_loss 0.5044814348220825


Epoch 4: 100%|██████████| 464/464 [03:54<00:00,  1.98it/s]

Epoch 4: train_loss 0.4983987510204315


In [5]:
trainer.test()

Validation: val_loss {'map': tensor(0.0246), 'map_50': tensor(0.0520), 'map_75': tensor(0.0209), 'map_small': tensor(0.0014), 'map_medium': tensor(0.0404), 'map_large': tensor(0.3236), 'mar_1': tensor(0.0232), 'mar_10': tensor(0.0385), 'mar_100': tensor(0.0391), 'mar_small': tensor(0.0092), 'mar_medium': tensor(0.0776), 'mar_large': tensor(0.3945), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=torch.int32)}
